# iMatreialist - Predictor

## TFNW

### @alkari

# Construct h5 file from raw images

In [ ]:
from __future__ import print_function
from __future__ import absolute_import

from keras.models import load_model
import numpy as np
import h5py, time, os
from PIL import Image

size = 224

In [ ]:
# Time
def elapsed (elapsed):
    hours, rem = divmod(elapsed, 3600)
    minutes, seconds = divmod(rem, 60)
    return("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

In [ ]:
images_path = 'test_images' # '/Users/alkari/Desktop/nasnet/test_images'

In [ ]:
os.chdir(images_path)
#!ls -lh

In [ ]:
filelist = []
for file in os.listdir('.')[:]:
    fname = file.split('.')
    if fname[1]=='jpg':
        image_id = int(fname[0])
        filelist.append(image_id)
    else:
        print('error '+file)

#print(len(filelist))

In [ ]:
start = time.time()
images = []
missing = []
for i in range(12801):
    
    if i in filelist:

        image = Image.open(str(i)+'.jpg') 
        # Ensure right mode
        if image.mode != 'RGB':
            image = image.convert('RGB')
        # Resize
        image = image.resize((size,size), resample=Image.LANCZOS)
        image = np.asarray(image)
        # preprocess
        image = image.astype('float32', copy=False)
        image /= 127.5
        image -=1        
        
        # if bad image, skip
        if image.shape != (size,size,3):
            print("Image "+str(image_id)+" size: "+str(image.shape)) 

    else:
        image = np.zeros((224,224,3))
        missing.append(i)

    # add image to images list
    images.append( image )


    if not i % 1000:
        print(str(i)+' images loaded in {}'.format(elapsed(time.time()-start)))

In [ ]:
with h5py.File('../eval_dataset.h5', 'w') as hf:
    hf.create_dataset('test_dataset',
                      data=images,
                      compression='gzip',
                      compression_opts=9,
                      chunks=(1000,224,224,3)
                     )
    hf.create_dataset('missing', data=missing)

# Predict and Create Submission

In [ ]:
from __future__ import print_function
from __future__ import absolute_import

from keras.models import load_model

import tensorflow as tf

import numpy as np
import pandas as pd
import h5py, time, os

model_path = '/Users/alkari/Documents/GoogleDrive/iMaterialist/iMaterialist-model.h5'
#trained_weights = '~/Documents/GoogleDrive/nasnet/weights/NASNet-iMaterialist (1).h5'

size = 224

In [ ]:
os.chdir('~/Desktop/')
os.makedirs('nasnet', exist_ok=True)
os.chdir('nasnet')

In [ ]:
!cp '~/Documents/GoogleDrive/nasnet/weights/NASNet-iMaterialist (1).h5' NASNet-iMaterialist.h5
!ls

In [ ]:
if tf.device('/CPU:0'):
  print('Loading model takes about 3 minutes...')
  model=load_model(model_path)
  print('Model loaded. Loading weights...')
  model.load_weights('NASNet-iMaterialist.h5')
  print('Weights loaded.')

In [ ]:
def load_final_testset(filename,max=None):
    with h5py.File(filename, 'r') as hf:
        test_images = hf['test_dataset'][:max]
        #test_images = preprocess_input(test_images)
        missing = hf['missing'][:]
    return test_images,missing

In [ ]:
test_file = '~/Desktop/nasnet/eval_dataset.h5'
x_test_final,missing = load_final_testset(test_file)
predictions = model.predict(x_test_final, verbose=1, batch_size=100)

In [ ]:
predictions = predictions[0].argmax(-1)

In [ ]:
# Adjust for files force labeled 0 to 128 and label missing files.
arbitrary = 7 # or random!
for i in range(predictions.shape[0]):
    if predictions[i]==0:
        predictions[i]=128
        print('Changed image {} from 0 to {}'.format(i,predictions[i]))
    if i in missing:
        predictions[i]=arbitrary
        print('Changed missing image {} prediction to {}'.format(i,predictions[i]))
        


In [ ]:
df = pd.DataFrame({'id' : range(len(predictions)), 
                   'predicted' : predictions})
df = df.drop(0) # Drop none existent image[0] prediction
if not os.path.isdir('predictions'):
    os.mkdir('predictions')
filename = 'predictions/tfnw-preds_' + time.strftime('%Y%m%d_%H%M%S_%Z.csv')
df.to_csv(filename, index=False)

In [ ]:
for i in range(10):
    print(i, predictions[i])

In [ ]:
!head $filename

In [ ]:
for i in range(predictions.shape[0]-10,predictions.shape[0]):
    print(i, predictions[i])

In [ ]:
!tail $filename

In [ ]:
df